### Built-in scope

In [ ]:
tuple = {"name": "john doe"}
print(tuple(0, 1, 2)) # Using built-in functions as identifiers masks default behavior.

### Global scope

In [ ]:
x = "global x" 

def level_one():
    return x

print(level_one()) # return global x

In [ ]:
'''

1. RULE: Lookups happens at runtime, location is decided at compile time.
2. No x in local scope, x in the global scope is used.
3. Avoid naming conflicts with namespaces, x is present in both local and global scope.'

'''

### Local scope

In [ ]:
x = "global x" 

def level_two(v):
    # print(x) # Trying to access x before assignment throws UnboundLocalError
    if v:
        x = "local x"
    return x

print(level_two(True)) # return local x

In [ ]:
print(level_two(False)) # throws UnboundLocalError

'''
1. Python compiles the function before execution and creates a function object.
2. Detects assignment statement and marks x as local variable.
3. If v = False, there is no assignment, hence the error UnboundLocalError
4. UnboundLocalError: cannot access local variable 'x' where it is not associated with a value'

'''

In [ ]:
def square(base):
    result = base ** 2
    print(f"Square of {base} : {result}")

# New L. scope for each function call.
square(20) # Square of 20 : 400

# New L. scope for each function call.
square(10) # Square of 10 : 100

In [ ]:
# Reuse variable names, cube's local scope has no knowledge about square's 
def cube(base):
    result = base ** 3
    print(f"Cube of {base} : {result}")

# Trying to access local var: result outside throws NameError.
print(result) # NameError: name 'result' is not defined

### Enclosing Scope

In [ ]:
x = "global x"

def level_three():
    z = "first outer z"

    def inner(y):
        return x, y, z
    
    z = "second outer z"
    print(inner.__closure__) # reference to enclosing func namespace
    print(inner.__globals__) # reference to global namespace
    return inner("y arg")

print(level_three()) # ('global x', 'y arg', 'second outer z')

In [ ]:
'''
In the end inner() is called.
1. y - passed as an arg, found in the local scope.
2. x - not in local and enclosing scope, found in global.
3. z - not defined in the local scope, found in the enclosing scope.
z is reassigned before invoking inner()
'''

In [ ]:
x = "global x"

def level_four():
    z = "outer z"
    def donky():
        def inner(y):
            return x, y, z
        return inner
    
    def chonky():
        x = "chonky x"
        f = donky()
        return f("y arg")
    
    return chonky()

print(level_four()) # ('global x', 'y arg', 'outer z')

In [ ]:
'''
# Execution flow
1. global - x, level_five
2. level_five is called, scope - z, donky, chonky, return - calls chonky
3. chonky - x, f -> calls donky -> inner, return - inner("y arg")
4. donky - inner, returns inner
5. inner: y - local var, z - found in enclosing scope, x - global scope
'''

In [ ]:
def level_five():
    
    def inner():
        if False:
            a = None
        
        def gen_func():
            nonlocal a
            for v in range(10):
                a = v
                yield v 
                
        return gen_func(), lambda: a
        
    gen, fun = inner()
    
    # print(fun()) # throws NameError, because a is not defined
    next(gen)
    print(fun())
    next(gen)
    print(fun())
    next(gen)
    print(fun())

level_five()

In [ ]:
'''
gen - has generator instance; fun - lambda func returns a
1. Calling fun() first throws NameError: cannot access free variable 'a' where it is not associated with a value in enclosing scope
2. a is present in enclosing sope but unassigned before gen_func()
3. Free Variable - variables belonging to enclosing scope, that are used in local scope.
4. next(gen) - return 0 and assigns a = 0
5. Lambda are also functions and are treated as functions.
6. Generator is also treated as functions.

'''

### Exception cases LEGB rule

In [ ]:
## 1. Comprehension

listComp = [i for i in range(1, 6)]

print(listComp)
# print(i) # NameError: name 'i' is not defined. 

for i in range(1, 6):
    print(i)

print(i) # 5

'''
# Accessing loop variable
1. Comprehension - result in NameError
2. for loop - returns the last iterator value.

'''

In [ ]:
## 2. try-except block
listComp = [i for i in range(1, 6)]
try:
    listComp[len(listComp)] 
except Exception as err:
    print(err) # list index out of range

print(err) # NameError: name 'err' is not defined
# exception variable is local to the except block

### Use of global and nonlocal statements

In [ ]:
x = "global x"
print(f"Global variables: {globals().get('x')}") # Global variables: global x

def global_and_nonlocal():
    x = "nonlocal x"
    print(f"Enclosing variables: {locals()}")  # {'x': 'nonlocal x'}

    def inner():
        global x
        x = "overwritten global x"
        print(f"Local Vars: {locals()}") # {} no L vars
    
    inner()

global_and_nonlocal()
print(x) # overwritten global x

In [ ]:
'''
1. globals() - returns global namespace
2. locals() - returns local namespace.
3. dir() - returns the objects in the current namespace.
4. nonlocal - refers to a variable in an enclosing function (but not global).
5. global - refers to a global variable, allowing modification.

'''

In [ ]:
def level_one():
    x = "nonlocal x"
    
    def level_two():
        nonlocal x
        x = "modified at level two"

        def level_three():
            nonlocal x
            x = "modified at level three"

        level_three()
        print(f"x after level three: {x}")

    level_two()
    print(f"x after level two: {x}")

level_one()


In [ ]:
# using global is considered as bad practice
# fix - pass immutable data types as arguments.
# mutable - pass a copy to avoid modifications.

url = "http://www.google.com"

# bad practice
def modify_url_bp():
    global url
    url = url.replace("http", "https")

# good practice
def modify_url_gp(link): # pass as parameter
    return link.replace("http", "https")

url = modify_url_gp(url)
print(url)


In [ ]:
counter = 0

def update_counter(count):
    return count + 1 

counter = update_counter(counter)
print(counter)
counter = update_counter(counter)
print(counter)
counter = update_counter(counter)
print(counter)